In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('/home/markaw/Downloads/diamonds_train.csv')
predict = pd.read_csv('/home/markaw/Downloads/diamonds_test.csv')

In [3]:
TARGET = 'price'
CAT_FEATURES = ['cut','color','clarity']
NUM_FEATURES = ['carat','depth','table','x','y','z']

for i in CAT_FEATURES:
    train[i] = train[i].astype('category')
    predict[i] = predict[i].astype('category')

In [4]:
categorical_train_df = pd.get_dummies(train[CAT_FEATURES])
numerical_train_df = train[NUM_FEATURES]
train_df = pd.concat([categorical_train_df, numerical_train_df],axis=1)

categorical_predict_df = pd.get_dummies(predict[CAT_FEATURES])
numerical_predict_df = predict[NUM_FEATURES]
predict_df = pd.concat([categorical_predict_df, numerical_predict_df],axis=1)

In [5]:
FEATURES = categorical_train_df.columns.tolist() + numerical_train_df.columns.tolist()

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean',fill_value= None)),
                                       ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant',
                                                                   fill_value= None)),
                                         ('econder', OneHotEncoder(handle_unknown='ignore'))])

categorical_transformer.fit_transform(train_df[FEATURES]).todense()

matrix([[1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.]])

In [12]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(transformers=[('numerical_preprocessor', numerical_transformer, NUM_FEATURES),
                                              ('categorical_preprocessor', categorical_transformer, CAT_FEATURES)])



In [13]:
from sklearn.preprocessing import RobustScaler


scaler = RobustScaler()
X = scaler.fit_transform(train_df.select_dtypes(include='number'))
y = train[TARGET]

In [14]:
from sklearn.linear_model import LinearRegression

model = Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', LinearRegression())])

X_train = train_df[FEATURES]
y_train = train[TARGET]


In [15]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import preprocessing


model= Pipeline(steps=[('regressor', RandomForestRegressor())])
model.fit(X_train,y_train)
X_predict = scaler.transform(predict_df[FEATURES])
predictions = model.predict(X_predict)

In [16]:
diamond_id = predict['id']
submission = pd.DataFrame({'id':diamond_id, 'price':predictions})
submission.to_csv('submission_csv',index=False)

## Model options

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()

model.fit(X,y)
X_predict = scaler.transform(predict_df[FEATURES])
predictions = model.predict(X_predict)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth = 4)

model.fit(X,y)
X_predict = scaler.transform(predict_df[FEATURES])
predictions = model.predict(X_predict)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model

lasso = linear_model.Lasso()
lasso.fit(X,y)
X_predict = scaler.transform(predict_df[FEATURES])
predictions = model.predict(X_predict)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import preprocessing


model= Pipeline(steps=[('regressor', RandomForestRegressor())])
model.fit(X,y)
X_predict = scaler.transform(predict_df[FEATURES])
predictions = model.predict(X_predict)